In [50]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

# 데이터 로드
books = pd.read_csv('Books.csv', encoding='utf-8')
filtered_ratings = pd.read_csv('Filtered_Ratings_NoZero.csv')

# transactions을 생성할 때 ISBN 값으로 리스트를 만들어주는 부분 수정
transactions = (
    filtered_ratings.groupby('User-ID')['ISBN']
    .apply(list)
    .reset_index(name='transactions')
)

# transactions을 one-hot encoding하여 DataFrame 생성
te = TransactionEncoder()
onehot = te.fit(transactions['transactions']).transform(transactions['transactions'])
transactions_onehot = pd.DataFrame(onehot, columns=te.columns_)

# A-Priori 알고리즘 적용
frequent_itemsets = apriori(transactions_onehot, min_support=0.01, use_colnames=True)

# Association Rule 생성
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.7)

# 사용자가 입력한 읽은 책
user_input_books = input("읽은 책을 ISBN으로 입력하세요 (여러 개의 ISBN은 쉼표로 구분): ").split(',')

# Association Rule을 이용하여 추천할 책 찾기
recommended_books = set()
for index, row in rules.iterrows():
    if row['antecedents'].issubset(user_input_books) and not row['consequents'].issubset(user_input_books):
        recommended_books.update(row['consequents'])

print(f"Association Rule에서 추출한 책들:\n{recommended_books}")

# 결과 출력
if recommended_books:
    recommended_books_titles = books.loc[books['ISBN'].isin(recommended_books), 'Book-Title'].values
    print(f"사용자에게 추천할 만한 책들:\n{recommended_books_titles}")
else:
    print("추천할 만한 책이 없습니다.")


C:\Users\asdf9\AppData\Local\Temp\ipykernel_14548\238123975.py:6: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('Books.csv', encoding='utf-8')


Association Rule에서 추출한 책들:
set()
추천할 만한 책이 없습니다.
